In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_interactions import ipyplot as iplt




In [ ]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
me.init_vprinting()
import control as ct
from control.matlab import *

t = me.dynamicsymbols._t
m, mw, Ixx, Iyy, Izz, h, g, a, b, delta = sm.symbols('m, mw, I_{xx}, I_{yy}, I_{zz}, h, g, a, b, delta')
Ixy, Ixz, Iyz = sm.symbols('I_{xy}, I_{xz}, I_{yz}')
theta, r, psi, y, x, z, xB, yB = me.dynamicsymbols('theta, r, psi, y, x, z, xB, yB')
q1, q2, q3, q4, u1, u2, u3, u4 = me.dynamicsymbols('q1, q2, q3, q4, u1, u2, u3, u4')
N, B, A, F = sm.symbols('N, B, A, F', cls=me.ReferenceFrame)
Bt, Ct, Dt, Et = sm.symbols('Bt, Ct, Dt, Et')
Bb, Cb, Db, Eb = sm.symbols('Bb, Cb, Db, Eb')

# Frames
A.orient_axis(N, r, N.z)# for positive turning left
B.orient_axis(A, -theta, A.x) # for lean to the left
F.orient_axis(B, delta, B.z)# for positive steer to the left

#q = sm.Matrix([theta, r, xB, yB, delta])# this should change since x,y,z,xB, yB can all relate to angles?
q1 = theta #roll
q2 = r #yaw
q3 = xB
q4 = yB

# these must be like this not theta = q1 otherwise the substitution gets weird
q = sm.Matrix([theta, r, xB, yB])

#q = sm.Matrix([theta, r, xB, yB])
qd = q.diff(t)
qdd = qd.diff(t)

u = sm.Matrix([u1, u2, u3, u4])# all generalized speeds
ud = u.diff(t)
ur = sm.Matrix([u3, u4]) # dependent 
us = sm.Matrix([u2]) # independent 
urd = ur.diff(t)
usd = us.diff(t)
p = sm.Matrix([Ixx, Iyy, Izz, Ixy, Ixz, Iyz, g, h, m, mw, t, a, b, Bt, Dt, Et])# constants in my equation

steer = sm.Matrix([delta])

#setting up my zeors
qd_zero = {qdi: 0 for qdi in qd} 
ur_zero = {uri: 0 for uri in ur} 
u_zero = {ui: 0 for ui in u}
urd_zero = {urdi: 0 for urdi in urd} 
usd_zero = {usdi: 0 for usdi in usd}
ud_zero = {udi: 0 for udi in ud}

######### Kinematic equations formulation ############
fk = sm.Matrix([
    u1 - q1.diff(t),
    u2 - q2.diff(t),
    u3 - q3.diff(t),
    u4 - q4.diff(t)])

### Solvinig the Kin eq for qd to change them to terms with u's #####
Mk = fk.jacobian(qd)
gk = fk.xreplace(qd_zero)
qd_sol = -Mk.LUsolve(gk)
qd_repl = dict(zip(qd, qd_sol))


### All of my points being set up ####
Base = me.Point('Base')
O = me.Point('O')
COM = me.Point('COM')
COMp = me.Point('COMp')
Wf = me.Point('Wf')
Wb = me.Point('Wb')

# telling python where my points are realtive to each other #
Wb.set_pos(O, q3*N.x + q4*N.y)#changes with time so they can be q's?
#Base.set_pos(Wb, (b*B.x))
COM.set_pos(Wb, b*B.x + h*B.z )#doesnt change with time
COMp.set_pos(COM, -h*sm.cos(q1)*N.z)#h.dot(-N.z)*N.z
Wf.set_pos(COM, (a*B.x - h*B.z))#a is fixed dist from COM to wheel gnd cont pnt
#Wb.set_pos(Base, -b*B.x+0*N.z)#b is a fixed dist from COM to wheel gnd cont pnt

B_w_F = F.ang_vel_in(B)

# getting velocities of COM #
O.set_vel(N, 0)
N_v_Wb = Wb.pos_from(O).diff(t, N)#need this for 2pt thing
Wb.set_vel(N, N_v_Wb)
N_v_COM = COM.v2pt_theory(Wb, N, B)

#N_v_Base = Base.v2pt_theory(Wb, N, B)
COM.set_vel(B,0)
N_v_Wf = Wf.v2pt_theory(COM, N, B)
## how would I give direction to this? 


 #this is U
N_v_COMp = COMp.v2pt_theory(Wb, N, N)
# N_v_Wf_steer = N_v_COM.cross(B_w_F)
# angular velocity of my body #
N_w_A = A.ang_vel_in(N)
N_w_B = B.ang_vel_in(N)#.xreplace(qd_repl)
B_w_F = F.ang_vel_in(B)

I_B = me.inertia(B, Ixx, Iyy, Izz, Ixy, Ixz, Iyz)
N_v_Wb.express(A)

loop1 = Wf.pos_from(O).express(N)
loop2 = Wb.pos_from(O)
Holo_const = Wf.pos_from(Wb).dot(B.z)



v_forward = N_v_Wf.dot(F.x) * sm.cos(delta) + N_v_Wf.dot(F.y) * sm.sin(delta)
v_sideways = -N_v_Wf.dot(F.x) * sm.sin(delta) + N_v_Wf.dot(F.y) * sm.cos(delta)

N_v_Wf


In [ ]:
# # calculating Inertia
# Ifront = (0.527*me.outer(N.x, N.x) +
#      1.317*me.outer(N.y, N.y) +
#      0.759*me.outer(N.z, N.z) -
#      -0.114*me.outer(N.x, N.z) -
#      -0.114*me.outer(N.z, N.x))

# Irear = (0.253*me.outer(N.x, N.x) +
#      1.317*me.outer(N.y, N.y) +
#      0.0956*me.outer(N.z, N.z) -
#      -0.072*me.outer(N.x, N.z) -
#      -0.072*me.outer(N.z, N.x))

# IrearW = (0.0904*me.outer(N.x, N.x) +
#      0.152*me.outer(N.y, N.y) +
#      0.0904*me.outer(N.z, N.z))

# IfrontW = (0.0884*me.outer(N.x, N.x) +
#      0.149*me.outer(N.y, N.y) +
#      0.0884*me.outer(N.z, N.z))

# F_W = me.Point('F_W')
# F_F = me.Point('F_F')
# R_W = me.Point('R_W')
# R_F = me.Point('R_F')

# F_F.set_pos(Wb, 0.867*B.x - 0.748*B.z)
# R_F.set_pos(Wb, 0.276*B.x - 0.538*B.z)
# F_W.set_pos(Wf, -0.34*B.z)#a is fixed dist from COM to wheel gnd cont pnt
# R_W.set_pos(Wb, -0.34*B.z)#b is a fixed dist from COM to wheel gnd cont pnt

# m_FW = 2.02
# m_RW = 3.11
# m_FF = 3.22
# m_RF = 9.86

# r_FF_COM = COM.pos_from(F_F)
# r_RF_COM = COM.pos_from(R_F)
# r_FW_COM = COM.pos_from(F_W)
# r_RW_COM = COM.pos_from(R_W)

# U = me.outer(B.x, B.x) + me.outer(B.y, B.y) + me.outer(B.z, B.z)

# I_FF_COM = m_FF*(me.dot(r_FF_COM, r_FF_COM)*U - me.outer(r_FF_COM, r_FF_COM))
# I_RF_COM = m_RF*(me.dot(r_RF_COM, r_RF_COM)*U - me.outer(r_RF_COM, r_RF_COM))
# I_FW_COM = m_FW*(me.dot(r_FW_COM, r_FW_COM)*U - me.outer(r_FW_COM, r_FW_COM))
# I_RW_COM = m_RW*(me.dot(r_RW_COM, r_RW_COM)*U - me.outer(r_RW_COM, r_RW_COM))

# I_FF = Ifront + I_FF_COM
# I_RF = Irear + I_RF_COM
# I_FW = IfrontW + I_FW_COM
# I_RW = IrearW + I_RW_COM

# I_B = I_FF + I_RF + I_FW + I_RW
# I_B
# r_FF_COM

In [ ]:
# # calculating Inertia with mass
# Ifront = (11.3*me.outer(N.x, N.x) +
#      12.218*me.outer(N.y, N.y) +
#      3.124*me.outer(N.z, N.z) -
#      -1.968*me.outer(N.x, N.z) -
#      -1.968*me.outer(N.z, N.x))

# Irear = (0.253*me.outer(N.x, N.x) +
#      0.246*me.outer(N.y, N.y) +
#      0.0956*me.outer(N.z, N.z) -
#      -0.072*me.outer(N.x, N.z) -
#      -0.072*me.outer(N.z, N.x))

# IrearW = (0.088*me.outer(N.x, N.x) +
#      0.152*me.outer(N.y, N.y) +
#      0.088*me.outer(N.z, N.z))

# IfrontW = (0.0*me.outer(N.x, N.x) +
#      0.149*me.outer(N.y, N.y) +
#      0.09*me.outer(N.z, N.z))

# F_W = me.Point('F_W')
# F_F = me.Point('F_F')
# R_W = me.Point('R_W')
# R_F = me.Point('R_F')

# F_F.set_pos(Wb, 0.867*B.x - 0.748*B.z)
# R_F.set_pos(Wb, 0.276*B.x - 1.04*B.z)
# F_W.set_pos(Wf, -0.34*B.z)#a is fixed dist from COM to wheel gnd cont pnt
# R_W.set_pos(Wb, -0.34*B.z)#b is a fixed dist from COM to wheel gnd cont pnt

# m_FW = 2.02
# m_RW = 3.11
# m_FF = 81.86
# m_RF = 3.22

# r_FF_COM = COM.pos_from(F_F)
# r_RF_COM = COM.pos_from(R_F)
# r_FW_COM = COM.pos_from(F_W)
# r_RW_COM = COM.pos_from(R_W)

# U = me.outer(B.x, B.x) + me.outer(B.y, B.y) + me.outer(B.z, B.z)

# I_FF_COM = m_FF*(me.dot(r_FF_COM, r_FF_COM)*U - me.outer(r_FF_COM, r_FF_COM))
# I_RF_COM = m_RF*(me.dot(r_RF_COM, r_RF_COM)*U - me.outer(r_RF_COM, r_RF_COM))
# I_FW_COM = m_FW*(me.dot(r_FW_COM, r_FW_COM)*U - me.outer(r_FW_COM, r_FW_COM))
# I_RW_COM = m_RW*(me.dot(r_RW_COM, r_RW_COM)*U - me.outer(r_RW_COM, r_RW_COM))

# I_FF = Ifront + I_FF_COM
# I_RF = Irear + I_RF_COM
# I_FW = IfrontW + I_FW_COM
# I_RW = IrearW + I_RW_COM

# I_B = I_FF + I_RF + I_FW + I_RW
# I_B

In [ ]:
N_v_Wb_nh = N_v_Wb.xreplace(qd_repl)
N_v_Wf_nh = N_v_Wf.xreplace(qd_repl)

nh = sm.Matrix([N_v_Wf_nh.dot(F.y),
                N_v_Wb_nh.dot(A.y)])# change the second to an intermediate frame
## the dot product is the magnitude of each vector times the cos of the angle between the vectors so it doesnt matter if it is pos or neg Y direction 

Mn = nh.jacobian(ur)
gn = nh.xreplace(ur_zero)
ur_sol = sm.trigsimp(-Mn.LUsolve(gn))
ur_repl = dict(zip(ur, ur_sol))

# finding the nonholo constraints as longitude and lateral slip
fnd = nh.diff(t).xreplace(qd_repl)# did qd_repl not work?
Mnd = fnd.jacobian(ud)
gnd = me.msubs(fnd, {u1.diff(t):0,u2.diff(t):0, u3.diff(t):0, u4.diff(t):0, delta.diff(t):0 })




# the non holo constraint (vel terms) and trying to get Mn, gn

In [ ]:
########## nonholo const of wheel ##########

# Calculate slip angles at the front and rear tires
alpha_f = delta - sm.atan2(N_v_Wf.dot(F.y), N_v_Wf.dot(F.x))
alpha_f = alpha_f.xreplace(qd_repl)
alpha_b = -sm.atan2(N_v_Wb.dot(A.y), N_v_Wb.dot(A.x))
alpha_b = alpha_b.xreplace(qd_repl)

# simple linear lateral force
Fyf = -Ct * alpha_f
Fyf = Fyf.xreplace(qd_repl)
Fyb = -Ct * alpha_b
Fyb = Fyb.xreplace(qd_repl)

# # Use Magic Formula to calculate lateral forces
# Fyf = Dt * sm.sin(Ct * sm.atan(Bt * alpha_f - Et * (Bt * alpha_f - sm.atan(Bt * alpha_f))))
# Fyb = Dt * sm.sin(Ct * sm.atan(Bt * alpha_b - Et * (Bt * alpha_b - sm.atan(Bt * alpha_b))))

Q1 = Fyf * Wf.pos_from(O).diff(q1, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q1, N).dot(A.y)
Q1 = Q1.xreplace(qd_repl)
Q2 = Fyf * Wf.pos_from(O).diff(q2, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q2, N).dot(A.y)
Q2 = Q2.xreplace(qd_repl)
Q3 = Fyf * Wf.pos_from(O).diff(q3, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q3, N).dot(A.y)
Q3 = Q3.xreplace(qd_repl)
Q4 = Fyf * Wf.pos_from(O).diff(q4, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q4, N).dot(A.y)
Q4 = Q4.xreplace(qd_repl)

Q = sm.Matrix([ Q1, Q2, Q3, Q4])


### add in steer
# my constraint saying there will be no movement in Y #
nh_new = sm.Matrix([N_v_Wf_nh.dot(F.y) - Fyf,
                    N_v_Wb_nh.dot(A.y) - Fyb])# change the second to an intermediate frame

Mn_new = nh_new.jacobian(ur)
gn_new = nh_new.xreplace(ur_zero)

# finding the nonholo constraints as longitude and lateral slip
fnd_new = nh_new.diff(t).xreplace(qd_repl)# did qd_repl not work?
Mnd_new = fnd_new.jacobian(ud)
gnd_new = me.msubs(fnd_new, {u1.diff(t):0,u2.diff(t):0, u3.diff(t):0, u4.diff(t):0, delta.diff(t):0 })



 # Finding KE, PE and Lagrange

In [ ]:
# KE EQ

KE_trans = m/2*(N_v_COM.dot(N_v_COM))
KE_rot = N_w_B.dot(I_B.dot(N_w_B))/2

KE_front = (mw / 2) * (N_v_Wf.dot(N_v_Wf))
KE_rear = (mw / 2) * (N_v_Wb.dot(N_v_Wb)) 

KE = KE_trans + KE_rot# + KE_front + KE_rear
KE= sm.Matrix([sm.trigsimp(KE)])# the I part rdot the book has r and the mass part seems wrong 


#Potential Energy
#PE=mgh
#heigth = COM.pos_from(O).express(N).dot(N.z)# <<me figuring out why it didnt work before
PE = sm.Matrix([m*g*-COM.pos_from(O).express(N).dot(N.z)])



# Update the equations of motion
qdd_zerod = {qddr: 0 for qddr in qdd}
fd_term_1 = KE.jacobian(qd).diff(t).transpose()
fd_term_2 = KE.jacobian(q).transpose()
fd_term_3 = PE.jacobian(q).transpose()
fd = fd_term_1 - fd_term_2 + fd_term_3
fd

# Compute the mass matrix and generalized forces for the updated system
Md = fd.jacobian(qdd)
gd = sm.trigsimp(fd.xreplace(qdd_zerod)).xreplace(qd_repl)

# Update the lagrange system with generalized forces
Md_lagrange = sm.Matrix([[Md, Mnd.transpose()], [Mnd, sm.zeros(2, 2)]])
gd_lagrange = sm.Matrix([gd, gnd])
### I need the lagrange multipliers now
#Md.free_symbols|gd.free_symbols
Md_lagrange

In [ ]:
KE_trans = m/2*(N_v_COM.dot(N_v_COM))
KE_rot = N_w_B.dot(I_B.dot(N_w_B))/2

KE_front = (mw / 2) * (N_v_Wf.dot(N_v_Wf))
KE_rear = (mw / 2) * (N_v_Wb.dot(N_v_Wb)) 

KE = KE_trans + KE_rot# + KE_front + KE_rear
KE= sm.trigsimp(KE)# the I part rdot the book has r and the mass part seems wrong 

#Potential Energy
#PE=mgh
#heigth = COM.pos_from(O).express(N).dot(N.z)# <<me figuring out why it didnt work before
PE = m*g*-COM.pos_from(O).express(N).dot(N.z)


#lagrange part
L = sm.Matrix([KE - PE])
sm.trigsimp(L)

fd = -(L.jacobian(qd).diff(t) - L.jacobian(q)).transpose()

# Compute the mass matrix and generalized forces for the updated system
Md = fd.jacobian(qdd)
gd = sm.trigsimp(fd.xreplace(qdd_zerod)).xreplace(qd_repl)

# Update the lagrange system with generalized forces
Md_lagrange = sm.Matrix([[Md, Mnd.transpose()], [Mnd, sm.zeros(2, 2)]])
gd_lagrange = sm.Matrix([gd, gnd])
### I need the lagrange multipliers now
#Md.free_symbols|gd.free_symbols

fd


In [ ]:
x = combined_matrix = sm.Matrix.vstack(q, qd, qdd)

q_eq = sm.Matrix([theta, r, xB, yB])
qd_eq = sm.Matrix([0,0,0,0])
qdd_eq = sm.Matrix([0,0,0,0])

xe = sm.Matrix.vstack(q_eq, qd_eq, qdd_eq)
xe_repl = dict(zip(x, xe))

### Taylor all at once ###
## first term ##
fd_eq = fd.xreplace(xe_repl)

## second term ##
fd_x = fd.jacobian(x)
x_xe = x - xe
fd_dx_eq = fd_x.applyfunc(lambda elem: elem.xreplace(xe_repl))

fd_dx_eq_sub = fd_dx_eq.subs({sm.sin(theta): theta, sm.cos(theta):1, sm.cos(r+theta):1, sm.cos(r-theta):1, sm.sin(r+theta): r+theta , sm.sin(r-theta) : r-theta , sm.cos(2*theta): 1, sm.sin(2*theta): 2*theta, sm.sin(r) : r, sm.cos(r):1 })

# Extracting the matrices
K_mat = fd_dx_eq_sub[:, 0:4]   # First 4 columns
C_mat = fd_dx_eq_sub[:, 4:8]   # Next 4 columns
M_mat = fd_dx_eq_sub[:, 8:12]  # Last 4 columns

M_mat

In [ ]:

KE_trans = m/2*(N_v_COM.dot(N_v_COM))
KE_rot = N_w_B.dot(I_B.dot(N_w_B))/2


N_v_Wf_s = Fyf*(F.y) + N_v_Wf
N_v_Wb_s = Fyf*(A.y) + N_v_Wb

KE_front = (mw / 2) * (N_v_Wf_s.dot(N_v_Wf_s))
KE_rear = (mw / 2) * (N_v_Wf_s.dot(N_v_Wf_s)) 

KE_new= KE_trans + KE_rot# + KE_front + KE_rear


#Potential Energy
#PE=mgh
#heigth = COM.pos_from(O).express(N).dot(N.z)# <<me figuring out why it didnt work before
PE_new = m*g*-COM.pos_from(O).express(N).dot(N.z)


# Update the equations of motion
#lagrange part
L_new = sm.Matrix([KE_new - PE_new])
sm.trigsimp(L)

fd_slip = -(L_new.jacobian(qd).diff(t) - L_new.jacobian(q)).transpose()

# #lagrange part
# L_new = sm.Matrix([KE_new - PE_new])

# # Update the equations of motion
# qdd_zerod = {qddr: 0 for qddr in qdd}
# fd_new = -(L_new.jacobian(qd).diff(t) - L_new.jacobian(q) ).transpose() + Q

# Compute the mass matrix and generalized forces for the updated system
Md_new = fd_slip.jacobian(qdd)
gd_new = fd_slip.xreplace(qdd_zerod).xreplace(qd_repl)


# Update the lagrange system with generalized forces
Md_lagrange_new = sm.Matrix([[Md_new, Mnd_new.transpose()], [Mnd_new, sm.zeros(2, 2)]])
gd_lagrange_new = sm.Matrix([gd_new, gnd_new])

Mnd_new

In [ ]:
x = sm.Matrix.vstack(q, qd, qdd)

q_eq = sm.Matrix([theta, r, xB, yB])
qd_eq = sm.Matrix([0,0,0,0])
qdd_eq = sm.Matrix([0,0,0,0])

xe = sm.Matrix.vstack(q_eq, qd_eq, qdd_eq)
xe_repl = dict(zip(x, xe))

### Taylor all at once ###
## first term ##
fd_eq = fd_slip.xreplace(xe_repl)

## second term ##
fd_x_slip = fd_slip.jacobian(x)
x_xe = x - xe
fd_dx_eq_slip = fd_x.applyfunc(lambda elem: elem.xreplace(xe_repl))


fd_dx_slip_sub = fd_dx_eq_slip.subs({sm.sin(theta): theta, sm.cos(theta):1, sm.cos(r+theta):1, sm.cos(r-theta):1, sm.sin(r+theta): r+theta , sm.sin(r-theta) : r-theta , sm.cos(2*theta): 1, sm.sin(2*theta): 2*theta, sm.sin(r) : r, sm.cos(r):1 })


# Extracting the matrices
K_mat_slip = fd_dx_slip_sub[:, 0:4]   # First 4 columns
C_mat_slip = fd_dx_slip_sub[:, 4:8]   # Next 4 columns
M_mat_slip = fd_dx_slip_sub[:, 8:12]  # Last 4 columns

M_mat_slip
K_mat_slip

#do the small angle things

In [ ]:
###### STATE SPACE ###########

# # # # # A # # # # 
A_mat_slip = sm.Matrix([[ sm.zeros(4, 4) , M_mat_slip],[M_mat_slip, C_mat_slip]])

# # # # # B # # # # 
B_mat_slip = sm.Matrix([[-M_mat_slip, sm.zeros(4, 4)],[sm.zeros(4, 4), K_mat_slip]])

z = sm.Matrix.vstack(q, qd)
A_mat_slip
B_mat_slip


# simulation setup:
 I need the MD,gd, Mk, gk and I think the Mnd, gnd

In [ ]:
import numpy as np
import math
#inputs
x0 = np.array([
    np.deg2rad(5), #q1 theta, lean
    np.deg2rad(0), #q2 r ang_vel
    0.1, #q3 x
    0.1, #q4 y
    np.deg2rad(1), #u1 theta
    np.deg2rad(10), #u2 r ang_acc
    3, #u3 x
    0.1 #u4 y 
])

p_vals = np.array([
    0.05, #Ixx
    2.0, #Iyy
    1.5, #Izz
    0.05,#Ixy
    0.02,#Ixz
    0.03,#Iyz
    9.8, #g
    5, #h
    81, #m
    1, #mw
    0.1, #t
    3, #a
    3, #b
    10, # Bt
    1, #Dt
    0.97 #Et
])

steer_vals = np.array([
    np.deg2rad(0) #delta
])

Ct_val = np.array([
    0.7
])


def roll_angle_event(t, x, p):
    """
    Event function to stop the solver if roll angle (theta) exceeds 25 or goes below -25 degrees.
    """
    roll_angle = x[0]  # q[0] represents the roll angle (theta)
    max_angle_rad = np.deg2rad(40)  # Convert 25 degrees to radians
    
    # Check if roll angle exceeds 25 degrees or is below -25 degrees
    if roll_angle > max_angle_rad:
        return roll_angle - max_angle_rad  # Positive value triggers the event
    elif roll_angle < -max_angle_rad:
        return roll_angle + max_angle_rad  # Negative value triggers the event
    return 1  # If no event, return a non-zero value (normal operation)

# Set the event properties
roll_angle_event.terminal = True  # Stop the solver when the event is triggered
roll_angle_event.direction = 0    # Detect event when crossing from either direction (positive or negative)



###### solving for my states
eval_lagrange = sm.lambdify((q, u, p, steer), [Mk, gk, Md_lagrange, gd_lagrange], cse=True)

def eval_rhs_lagrange(t,  x, p, steer):
    q = x[0:4]
    u = x[4:8]
    roll_angle = q[0]
    K = (math.sqrt(u[3]**2 +u[2]**2))/(p[6]*(p[11]+p[12]))
    steer_angle = [-K * roll_angle] # this instead of a 2 should be the equation relating steer to the lean angle
  

    Mk_vals, gk_vals, Md_lagrange, gd_lagrange = eval_lagrange(q, u, p, steer_angle)
    
    qd = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
    ud_and_lam = np.linalg.solve(-Md_lagrange, np.squeeze(gd_lagrange))
    
    ud = ud_and_lam[0:4]
    lam = ud_and_lam[4:6]
    
  
    return np.hstack((qd, ud)), lam, K


t0, tf = 0.0, 100.0
fps = 30
ts = np.linspace(t0, tf, num=int((tf-t0)*fps))

sol_lagrange = solve_ivp(
    lambda t, x, p: eval_rhs_lagrange(t, x, p, steer)[0],
    (t0, tf), x0, args=(p_vals,), t_eval=ts, events=roll_angle_event
)



In [ ]:

# ### python control.ss must be in numpy not symbolic ###
# A_slip = sm.lambdify((q, p), [A_mat_slip], cse=True)
# B_slip = sm.lambdify((q, p), [B_mat_slip], cse=True)

# A_slip_num = A_slip(x0[0:4], p_vals)
# B_slip_num = B_slip(x0[0:4], p_vals)

# C_mat = np.array([[1,0,0,0,0,0,0,0,0,0,0,0]])
# D_mat = np.array([[0]])

# import control
# # Create state-space system
# system = control.ss(A_slip_num, B_slip_num, C_mat, D_mat)
# ## controls only accepts float


In [ ]:
Md_lagrange_new, gd_lagrange_new

In [ ]:
eval_lagrange_slip = sm.lambdify((q, u, p, steer, Ct), [Mk, gk, Md_lagrange_new, gd_lagrange_new], cse=True)

# Lambdifying the slip angle equations
eval_alpha_f = sm.lambdify((q, u, p, steer), alpha_f, cse=True)
eval_alpha_b = sm.lambdify((q, u, p, steer), alpha_b, cse=True)

eval_Fyf = sm.lambdify((q, u, p, steer, Ct), Fyf, cse=True)
eval_Fyb = sm.lambdify((q, u, p, steer, Ct), Fyb, cse=True)


def eval_rhs_lagrange_slip(t, x, p, steer, Ct):
    q = x[0:4]
    u = x[4:8]
    roll_angle = q[0]
    K = (math.sqrt(u[3]**2 +u[2]**2))/(p[6]*(p[11]+p[12]))
    steer_angle = [-K * roll_angle] # this instead of a 2 should be the equation relating steer to the lean angle
    Ct = 0.7

    # if 5 <= t <= 20:
    #     #Ct = 0.013  # Slip condition between t=5 and t=10
    #     Ct = 200
    # else:
    #     Ct = 0.7 
    # Now evaluate the system with the updated Ct as a separate input
    Mk_vals_slip, gk_vals_slip, Md_lagrange_slip, gd_lagrange_slip = eval_lagrange_slip(q, u, p, steer_angle, Ct)
    
    qd_slip = np.linalg.solve(-Mk_vals_slip, np.squeeze(gk_vals_slip))
    ud_and_lam_slip = np.linalg.solve(-Md_lagrange_slip, np.squeeze(gd_lagrange_slip))
    
    ud_slip = ud_and_lam_slip[0:4]
    lam_slip = ud_and_lam_slip[4:6]

    # Compute the slip angles using the lambdified functions
    alpha_f_val = eval_alpha_f(q, u, p, steer_angle)
    alpha_b_val = eval_alpha_b(q, u, p, steer_angle)

    # Compute the lateral force 
    Fyf_val = eval_Fyf(q, u, p, steer_angle, Ct)
    Fyb_val = eval_Fyb(q, u, p, steer_angle, Ct)



    print(Ct)
    print (Fyf_val)
    
    
    return np.hstack((qd_slip, ud_slip)), lam_slip, alpha_f_val, alpha_b_val

t0, tf = 0.0, 100.0
fps = 30
ts = np.linspace(t0, tf, num=int((tf-t0)*fps))

sol_lagrange_slip = solve_ivp(
    lambda t, x, p: eval_rhs_lagrange_slip(t, x, p, steer, Ct)[0],
    (t0, tf), x0, args=(p_vals,), t_eval=ts, events=roll_angle_event
)



In [ ]:
###### solving for my states
eval_lagrange_new = sm.lambdify((q, u, p, steer, Ct), [Mk, gk, Md_lagrange_new, gd_lagrange_new], cse=True)
# Lambdifying the slip angle equations
eval_alpha_f = sm.lambdify((q, u, p, steer), alpha_f, cse=True)
eval_alpha_b = sm.lambdify((q, u, p, steer), alpha_b, cse=True)

def eval_rhs_lagrange_new(t, x, p, steer):
    q = x[0:4]
    u = x[4:8]
    roll_angle = q[0]
    K = (math.sqrt(u[3]**2 +u[2]**2))/(p[6]*(p[11]+p[12]))
    steer_angle = [-K * roll_angle] # this instead of a 2 should be the equation relating steer to the lean angle
  
    #Ct = 0.14

    # #Set Ct dynamically based on the conditions
    # #x[2] is xB and x[3] is yB
    # if  -20 <= x[2] <= -10 and 0 <= x[3] <= 20:
    #     Ct = 0.0002  # ice was between 0.15 and 0.25 so split the diff
    # else:
    #     Ct = 0.2  # from effects of winter road maint. by wallman 1997 this was 0.8 to 0.9 so split

    # Set Ct dynamically based on the time (t)
    if 30 <= t <= 40:
        Ct = 0.13  # Slip condition between t=5 and t=10
    else:
        Ct = 0.7  # Normal condition outside this time range
### C is about 110 N/deg assuming vert load on each tire, values btwn .9 and 1.4 can be concidered
# find percentage of Ct of car vs bike the implement that 

    # Now evaluate the system with the updated Ct as a separate input
    Mk_vals, gk_vals, Md_lagrange_new, gd_lagrange_new = eval_lagrange_new(q, u, p, steer_angle, Ct)
    
    qd = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
    ud_and_lam_new = np.linalg.solve(-Md_lagrange_new, np.squeeze(gd_lagrange_new))
    
    ud_new = ud_and_lam_new[0:4]
    lam_new = ud_and_lam_new[4:6]

    # Compute the slip angles using the lambdified functions
    alpha_f_val = eval_alpha_f(q, u, p, steer_angle)
    alpha_b_val = eval_alpha_b(q, u, p, steer_angle)
    
    return np.hstack((qd, ud_new)), lam_new, alpha_f_val, alpha_b_val


t0, tf = 0.0, 100.0
fps = 30
ts = np.linspace(t0, tf, num=int((tf-t0)*fps))

# sol_lagrange_new = solve_ivp(
#     lambda t, x, p: eval_rhs_lagrange_new(t, x, p, steer)[0],
#     (t0, tf), x0, args=(p_vals,), t_eval=ts
# )

# Initialize lists to store the alpha_f and alpha_b values
alpha_f_vals = []
alpha_b_vals = []
time_vals = []

# Define a custom lambda function for integration that also stores slip angles
def rhs_with_slip(t, x, p, steer):
    # Compute the state derivatives, lambdas, and slip angles
    dx, lam, alpha_f, alpha_b = eval_rhs_lagrange_new(t, x, p, steer)
    
    # Store slip angles for later plotting
    alpha_f_vals.append(alpha_f)
    alpha_b_vals.append(alpha_b)
    time_vals.append(t)
    
    # Return state derivatives (required by solve_ivp)
    return dx


# # Now run the solver with the event function
# sol_lagrange_new = solve_ivp(
#     lambda t, x: rhs_with_slip(t, x, p, steer),
#     (t0, tf), x0, t_eval=ts,
#     events=roll_angle_event  # Add the event function here
# )
sol_lagrange_new = solve_ivp(
    lambda t, x, p: rhs_with_slip(t, x, p, steer)[0],
    (t0, tf), x0, args=(p_vals,), t_eval=ts, events=roll_angle_event
)
# Check if an event occurred
if sol_lagrange_new.status == 1:
    print(f"Solver stopped at t={sol_lagrange_new.t_events[0]} because roll was over the limit.")


# # Run the simulation
# sol_lagrange_new = solve_ivp(
#     lambda t, x: rhs_with_slip(t, x, p_vals),
#     (t0, tf), x0, t_eval=ts
# )


In [ ]:
# # Function to extract positions from solutions
# def extract_positions_from_solution(solution, p_vals):
#     """Extract the positions of the Front Wheel, Rear Wheel, and Center of Mass (COM)."""
#     x_vals, y_vals = [], []
    
#     # Loop over all time points and extract positions
#     for i in range(solution.y.shape[1]):
#         x, y, z = eval_point_coords(solution.y.T[i, :4], p_vals)
#         x_vals.append([x[1], x[2], x[3]])  # Front wheel, Rear wheel, COM
#         y_vals.append([y[1], y[2], y[3]])
    
#     return np.array(x_vals), np.array(y_vals)

# # Extracting positions for both "With Slip" and "No Slip" solutions
# x_vals_slip, y_vals_slip = extract_positions_from_solution(sol_lagrange_slip, p_vals)
# x_vals_ice, y_vals_ice = extract_positions_from_solution(sol_lagrange_new, p_vals)

# # Plotting the results
# plt.figure(figsize=(10, 8))

# # Plot paths for "With Slip" (dashed lines)
# plt.plot(x_vals_slip[:, 0], y_vals_slip[:, 0], 'r--', label="Front Wheel Path (Slip)")
# plt.plot(x_vals_slip[:, 1], y_vals_slip[:, 1], 'm--', label="Rear Wheel Path ( Slip)")
# plt.plot(x_vals_slip[:, 2], y_vals_slip[:, 2], 'b--', label="COM Path ( Slip)")

# # Plot paths for "No Slip" (solid lines)
# plt.plot(x_vals_ice[:, 0], y_vals_ice[:, 0], 'g-', label="Front Wheel Path (Ice)")
# plt.plot(x_vals_ice[:, 1], y_vals_ice[:, 1], 'c-', label="Rear Wheel Path (Ice)")
# plt.plot(x_vals_ice[:, 2], y_vals_ice[:, 2], 'y-', label="COM Path (Ice)")

# # Add legend and labels
# plt.xlabel("X Position")
# plt.ylabel("Y Position")
# plt.title("Comparison of Paths With and Without Slip (Front Wheel, Rear Wheel, COM)")
# plt.legend()
# plt.grid(True)
# plt.axis('equal')
# plt.show()


In [ ]:
# # Convert alpha_f_vals, alpha_b_vals, and time_vals to numpy arrays
# alpha_f_vals = np.array(alpha_f_vals).flatten()
# alpha_b_vals = np.array(alpha_b_vals).flatten()
# time_vals = np.array(time_vals)

# from scipy.interpolate import interp1d

# # Create interpolation functions for alpha_f and alpha_b over time
# interp_alpha_f = interp1d(time_vals, alpha_f_vals, kind='linear', fill_value="extrapolate")
# interp_alpha_b = interp1d(time_vals, alpha_b_vals, kind='linear', fill_value="extrapolate")

# # Convert slip angles from radians to degrees
# alpha_f_deg = np.degrees(alpha_f_interp)
# alpha_b_deg = np.degrees(alpha_b_interp)

# # Create a figure with 2 subplots (stacked vertically)
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# # Plot the front slip angle in degrees on the first subplot
# ax1.plot(ts, alpha_f_deg, label='Slip Angle (Front)', color='b')
# ax1.set_xlabel('Time [s]')
# ax1.set_ylabel('Slip Angle [deg]')
# ax1.set_title('Front Slip Angle Over Time (Degrees)')
# ax1.grid(True)
# ax1.legend()

# # Plot the rear slip angle in degrees on the second subplot
# ax2.plot(ts, alpha_b_deg, label='Slip Angle (Rear)', color='r', linestyle='--')
# ax2.set_xlabel('Time [s]')
# ax2.set_ylabel('Slip Angle [deg]')
# ax2.set_title('Rear Slip Angle Over Time (Degrees)')
# ax2.grid(True)
# ax2.legend()

# # Adjust layout so that the plots don't overlap
# plt.tight_layout()

# # Show the plots
# plt.show()


In [ ]:
def euler_integrate(rhs_func, tspan, x0, p_vals, steer_vals, delt=0.03):
    """Returns state trajectory and corresponding values of time resulting
    from integrating the ordinary differential equations with Euler's
    Method.

    Parameters
    ==========
    rhs_func : function
       Python function that evaluates the derivative of the state and takes
       this form ``dxdt = f(t, x, p)``.
    tspan : 2-tuple of floats
       The initial time and final time values: (t0, tf).
    x0_vals : array_like, shape(2*n,)
       Values of the state x at t0.
    p_vals : array_like, shape(o,)
       Values of constant parameters.
    delt : float
       Integration time step in seconds/step.

    Returns
    =======
    ts : ndarray(m, )
       Monotonically equally spaced increasing values of time spaced apart
       by ``delt``.
    xs : ndarray(m, 2*n)
       State values at each time in ts.

    """
    # generate monotonically increasing values of time.
    duration = tspan[1] - tspan[0]
    num_samples = round(duration/delt) + 1
    ts = np.arange(tspan[0], tspan[0] + delt*num_samples, delt)

    # create an empty array to hold the state values.
    x = np.empty((len(ts), len(x0_vals)))

    # set the initial conditions to the first element.
    x[0, :] = x0_vals

    # use a for loop to sequentially calculate each new x.
    for i, ti in enumerate(ts[:-1]):
        x[i + 1, :] = x[i, :] + delt*rhs_func(ti, x[i, :], p_vals, steer_vals)

    return ts, x

In [ ]:
coordinates = O.pos_from(O).to_matrix(N)
for point in [Wf, Wb, COM]:
    coordinates = coordinates.row_join(point.pos_from(O).to_matrix(N))

eval_point_coords = sm.lambdify((q, p), coordinates)
eval_point_coords(x0[0:4], p_vals)



########## the ones with slip ###########

coordinates_new = O.pos_from(O).to_matrix(N)
for point in [Wf, Wb, COM]:
    coordinates_new = coordinates_new.row_join(point.pos_from(O).to_matrix(N))

eval_point_coords_new = sm.lambdify((q, p), coordinates_new)
eval_point_coords_new(x0[0:4], p_vals)


In [ ]:

# # initial configuration of the points
# x, y, z = eval_point_coords(x0[:4], p_vals)

# # create a figure
# fig = plt.figure()
# fig.set_size_inches((10.0, 10.0))

# # setup the subplots
# ax_3d = fig.add_subplot(1, 1, 1, projection='3d')

# # 3d view
# lines_3d, = ax_3d.plot(x, y, z)
# ax_3d.set_xlim((-20.0, 20.0))
# ax_3d.set_ylim((-20.0, 20.0))
# ax_3d.set_zlim((-20.0, 20.0))
# ax_3d.set_xlabel('x')
# ax_3d.set_ylabel('y')
# ax_3d.set_zlabel('z')

# fig.tight_layout()

# plt.close()

# # Animation 
# from matplotlib.animation import FuncAnimation

# def animate(i):
#     x, y, z = eval_point_coords(sol_lagrange.y.T[i, :4], p_vals)
#     lines_3d.set_data_3d(x, y, z)

# ani = FuncAnimation(fig, animate, len(sol_lagrange.t))

# from IPython.display import HTML
# import matplotlib
# matplotlib.rcParams['animation.embed_limit'] = 400.0
# HTML(ani.to_jshtml(fps=fps))

In [ ]:
# # initial configuration of the points
# x, y, z = eval_point_coords_new(x0[:4], p_vals)

# # create a figure
# fig = plt.figure()
# fig.set_size_inches((10.0, 10.0))

# # setup the subplots
# ax_3d = fig.add_subplot(1, 1, 1, projection='3d')

# # 3d view
# lines_3d, = ax_3d.plot(x, y, z)
# ax_3d.set_xlim((-20.0, 20.0))
# ax_3d.set_ylim((-20.0, 20.0))
# ax_3d.set_zlim((-20.0, 20.0))
# ax_3d.set_xlabel('x')
# ax_3d.set_ylabel('y')
# ax_3d.set_zlabel('z')

# fig.tight_layout()

# plt.close()

# # Animation 
# from matplotlib.animation import FuncAnimation
# skip_factor = 15

# def animate(i):
#     # Process every frame, but only update the lines for every nth frame
#     if i % skip_factor == 0:  # Update every 15th frame
#         frame_index = i  # Use the actual index for rendering
#         if frame_index >= len(sol_lagrange.t):
#             return  # In case the number of frames exceeds available data

#     x, y, z = eval_point_coords_new(sol_lagrange_new.y.T[i, :4], p_vals)
#     lines_3d.set_data_3d(x, y, z)

# # Modify the number of frames for the animation to account for skipping
# num_frames = len(sol_lagrange_new.t) 

# ani = FuncAnimation(fig, animate, frames=num_frames)


# from IPython.display import HTML
# import matplotlib
# matplotlib.rcParams['animation.embed_limit'] = 400.0
# HTML(ani.to_jshtml(fps=fps))

In [ ]:
def extract_positions(solution, a, b):
    """Extract the positions of the COM, Wf, and Wb from the solution, relative to the origin O."""
    
    theta_vals = solution.y[0]  # Roll angle (θ)
    r_vals = solution.y[1]      # Steering angle (r)
    xO_vals = solution.y[2]     # x position of origin O
    yO_vals = solution.y[3]     # y position of origin O

    # Calculate the positions of COM
    xB_vals = xO_vals + a * np.cos(theta_vals)  # x-position of COM
    yB_vals = yO_vals + a * np.sin(theta_vals)  # y-position of COM

    # Calculate the positions of Wf and Wb
    Wf_x = xO_vals + (a + b) * np.cos(theta_vals + r_vals)  # x-position of Wf
    Wf_y = yO_vals + (a + b) * np.sin(theta_vals + r_vals)  # y-position of Wf
    
    Wb_x = xO_vals - b * np.cos(theta_vals)  # x-position of Wb
    Wb_y = yO_vals - b * np.sin(theta_vals)  # y-position of Wb
    
    return xO_vals, yO_vals, Wf_x, Wf_y, Wb_x, Wb_y


# Extracting positions for the no-slip scenario
xB_no_slip, yB_no_slip, Wf_x_no_slip, Wf_y_no_slip, Wb_x_no_slip, Wb_y_no_slip = extract_positions(sol_lagrange, 3, 3)

# Plotting the results for the no-slip scenario only
plt.figure(figsize=(10, 8))

# Plot for COM (Center of Mass)
plt.plot(xB_no_slip, yB_no_slip, 'b-', label="COM Path (No Slip)")

# Plot for Wf (Front Wheel)
plt.plot(Wf_x_no_slip, Wf_y_no_slip, 'g-', label="Front Wheel Path (No Slip)")

# Plot for Wb (Rear Wheel)
plt.plot(Wb_x_no_slip, Wb_y_no_slip, 'c-', label="Rear Wheel Path (No Slip)")

# Add legend and labels
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Paths Without Slip")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()


In [ ]:

def extract_positions(solution, a, b):
    """Extract the positions of the COM, Wf, and Wb from the solution, relative to the origin O."""
    
    theta_vals = solution.y[0]  # Roll angle (θ)
    r_vals = solution.y[1]      # Steering angle (r)
    xO_vals = solution.y[2]     # x position of origin O
    yO_vals = solution.y[3]     # y position of origin O

    # Calculate the positions of COM
    xB_vals = xO_vals + a * np.cos(theta_vals)  # x-position of COM
    yB_vals = yO_vals + a * np.sin(theta_vals)  # y-position of COM

    # Calculate the positions of Wf and Wb
    Wf_x = xO_vals + (a + b) * np.cos(theta_vals + r_vals)  # x-position of Wf
    Wf_y = yO_vals + (a + b) * np.sin(theta_vals + r_vals)  # y-position of Wf
    
    Wb_x = xO_vals - b * np.cos(theta_vals)  # x-position of Wb
    Wb_y = yO_vals - b * np.sin(theta_vals)  # y-position of Wb
    
    return xO_vals, yO_vals, Wf_x, Wf_y, Wb_x, Wb_y


# Extracting positions for both scenarios
xB_no_slip, yB_no_slip, Wf_x_no_slip, Wf_y_no_slip, Wb_x_no_slip, Wb_y_no_slip = extract_positions(sol_lagrange, 3, 3)
xB_with_slip, yB_with_slip, Wf_x_with_slip, Wf_y_with_slip, Wb_x_with_slip, Wb_y_with_slip = extract_positions(sol_lagrange_slip, 3, 3)

# Plotting the results
plt.figure(figsize=(10, 8))

# Plot for COM
plt.plot(xB_no_slip, yB_no_slip, 'b-', label="COM Path (No Slip)")
plt.plot(xB_with_slip, yB_with_slip, 'r--', label="COM Path (With Slip)")

# Plot for Wf
plt.plot(Wf_x_no_slip, Wf_y_no_slip, 'g-', label="Front Wheel Path (No Slip)")
plt.plot(Wf_x_with_slip, Wf_y_with_slip, 'y--', label="Front Wheel Path (With Slip)")

# Plot for Wb
plt.plot(Wb_x_no_slip, Wb_y_no_slip, 'c-', label="Rear Wheel Path (No Slip)")
plt.plot(Wb_x_with_slip, Wb_y_with_slip, 'm--', label="Rear Wheel Path (With Slip)")

# Add legend and labels
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Comparison of Paths With and Without Slip")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

In [ ]:
# Function to extract positions from solutions
def extract_positions_from_solution(solution, p_vals):
    """Extract the positions of the Front Wheel, Rear Wheel, and Center of Mass (COM)."""
    x_vals, y_vals = [], []
    
    # Loop over all time points and extract positions
    for i in range(solution.y.shape[1]):
        x, y, z = eval_point_coords(solution.y.T[i, :4], p_vals)
        x_vals.append([x[1], x[2], x[3]])  # Front wheel, Rear wheel, COM
        y_vals.append([y[1], y[2], y[3]])
    
    return np.array(x_vals), np.array(y_vals)

# Extracting positions for both "With Slip" and "No Slip" solutions
x_vals_slip, y_vals_slip = extract_positions_from_solution(sol_lagrange_slip, p_vals)
x_vals_ice, y_vals_ice = extract_positions_from_solution(sol_lagrange_new, p_vals)

# Plotting the results
plt.figure(figsize=(10, 8))

# Plot paths for "With Slip" (dashed lines)
plt.plot(x_vals_slip[:, 0], y_vals_slip[:, 0], 'r--', label="Front Wheel Path (Slip)")
plt.plot(x_vals_slip[:, 1], y_vals_slip[:, 1], 'm--', label="Rear Wheel Path ( Slip)")
plt.plot(x_vals_slip[:, 2], y_vals_slip[:, 2], 'b--', label="COM Path ( Slip)")

# Plot paths for "No Slip" (solid lines)
plt.plot(x_vals_ice[:, 0], y_vals_ice[:, 0], 'g-', label="Front Wheel Path (Ice)")
plt.plot(x_vals_ice[:, 1], y_vals_ice[:, 1], 'c-', label="Rear Wheel Path (Ice)")
plt.plot(x_vals_ice[:, 2], y_vals_ice[:, 2], 'y-', label="COM Path (Ice)")

# Add legend and labels
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Comparison of Paths With and Without Slip (Front Wheel, Rear Wheel, COM)")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()





In [ ]:
# Function to extract positions from solutions
def extract_positions_from_solution(solution, p_vals):
    """Extract the positions of the Front Wheel, Rear Wheel, and Center of Mass (COM)."""
    x_vals, y_vals = [], []
    
    # Loop over all time points and extract positions
    for i in range(solution.y.shape[1]):
        x, y, z = eval_point_coords(solution.y.T[i, :4], p_vals)
        x_vals.append([x[1], x[2], x[3]])  # Front wheel, Rear wheel, COM
        y_vals.append([y[1], y[2], y[3]])
    
    return np.array(x_vals), np.array(y_vals)

# Extracting positions for both "With Slip" and "No Slip" solutions
x_vals, y_vals = extract_positions_from_solution(sol_lagrange, p_vals)
x_vals_slip, y_vals_slip = extract_positions_from_solution(sol_lagrange_slip, p_vals)

# Plotting the results
plt.figure(figsize=(10, 8))

# Plot paths for "Without Slip" (dashed lines)
plt.plot(x_vals[:, 0], y_vals[:, 0], 'r--', label="Front Wheel Path (No Slip)")
plt.plot(x_vals[:, 1], y_vals[:, 1], 'm--', label="Rear Wheel Path (No  Slip)")
plt.plot(x_vals[:, 2], y_vals[:, 2], 'b--', label="COM Path (No Slip)")

# Plot paths for "No Slip" (solid lines)
plt.plot(x_vals_slip[:, 0], y_vals_slip[:, 0], 'g-', label="Front Wheel Path (With Slip)")
plt.plot(x_vals_slip[:, 1], y_vals_slip[:, 1], 'c-', label="Rear Wheel Path (With Slip)")
plt.plot(x_vals_slip[:, 2], y_vals_slip[:, 2], 'y-', label="COM Path (With Slip)")

# Add legend and labels
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Comparison of Paths With and Without Slip (Front Wheel, Rear Wheel, COM)")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()



In [ ]:

# Time points from the solution
time_points = sol_lagrange.t

# Extract the state variables (q and u) from the solution
q1_vals = sol_lagrange.y[0, :]  # q1 = theta (roll angle)
q2_vals = sol_lagrange.y[1, :]  # q2 = r (yaw angle)
q3_vals = sol_lagrange.y[2, :]  # q3 = xB (x position of base)
q4_vals = sol_lagrange.y[3, :]  # q4 = yB (y position of base)

u1_vals = sol_lagrange.y[4, :]  # u1 = roll rate
u2_vals = sol_lagrange.y[5, :]  # u2 = yaw rate

# Compute the steer angle delta and its rate (d(delta)/dt) for the original solution
delta_vals = -2 * q1_vals  # Steer angle over time (original)
delta_dot_vals = np.gradient(delta_vals, time_points)  # Steer rate (time derivative)

# Convert all angles (q1, q2, delta) and angular rates (u1, u2, delta_dot) from radians to degrees
q1_vals_deg = np.degrees(q1_vals)
q2_vals_deg = np.degrees(q2_vals)
delta_vals_deg = np.degrees(delta_vals)
delta_dot_vals_deg = np.degrees(delta_dot_vals)

u1_vals_deg = np.degrees(u1_vals)
u2_vals_deg = np.degrees(u2_vals)

# Activate interactive mode
%matplotlib widget

import matplotlib.pyplot as plt
from mpl_interactions import ipyplot as iplt

# Plot each generalized coordinate q1 over time (in degrees)
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q1_vals_deg, ax=ax, label='Roll Angle (q1)')
ax.set_title("q1 (theta - Roll)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q1 (deg)")
ax.legend()
plt.show()

# Plot q2 (yaw angle) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q2_vals_deg, ax=ax, label='Yaw Angle (q2)')
ax.set_title("q2 (r - Yaw)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q2 (deg)")
ax.legend()
plt.show()

# Plot q3 (xB position) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q3_vals, ax=ax, label='X Position (q3)')
ax.set_title("q3 (xB - X Position)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q3 (m)")
ax.legend()
plt.show()

# Plot q4 (yB position) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q4_vals, ax=ax, label='Y Position (q4)')
ax.set_title("q4 (yB - Y Position)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q4 (m)")
ax.legend()
plt.show()

# Plot roll rate (u1) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, u1_vals_deg, ax=ax, label='Roll Rate (u1)')
ax.set_title("u1 (Roll Rate)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("u1 (deg/s)")
ax.legend()
plt.show()

# Plot yaw rate (u2) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, u2_vals_deg, ax=ax, label='Yaw Rate (u2)')
ax.set_title("u2 (Yaw Rate)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("u2 (deg/s)")
ax.legend()
plt.show()


# Plot Steer angle (delta) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, delta_vals_deg, ax=ax, label='Steer Angle (delta)')
ax.set_title("Steer Angle (delta)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Steer Angle (deg)")
ax.legend()
plt.show()

# Plot Steer rate (delta_dot) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, delta_dot_vals_deg, ax=ax, label='Steer Rate (delta_dot)')
ax.set_title("Steer Rate (delta_dot)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Steer Rate (deg/s)")
ax.legend()
plt.show()


In [ ]:
# Time points from both solutions
time_points = sol_lagrange.t
time_point_slip = sol_lagrange_slip.t
time_points_new = sol_lagrange_new.t


# Extract the state variables (q and u) from both solutions
q1_vals = sol_lagrange.y[0, :]  # q1 = theta (roll angle)
q2_vals = sol_lagrange.y[1, :]  # q2 = r (yaw angle)
q3_vals = sol_lagrange.y[2, :]  # q3 = xB (x position of base)
q4_vals = sol_lagrange.y[3, :]  # q4 = yB (y position of base)

u1_vals = sol_lagrange.y[4, :]  # u1 = roll rate
u2_vals = sol_lagrange.y[5, :]  # u2 = yaw rate
u3_vals = sol_lagrange.y[6, :]  # u3 = xB velocity
u4_vals = sol_lagrange.y[7, :]  # u4 = yB velocity


# Extract the state variables (q and u) from both solutions
q1_vals_slip = sol_lagrange_slip.y[0, :]  # q1 = theta (roll angle)
q2_vals_slip = sol_lagrange_slip.y[1, :]  # q2 = r (yaw angle)
q3_vals_slip = sol_lagrange_slip.y[2, :]  # q3 = xB (x position of base)
q4_vals_slip = sol_lagrange_slip.y[3, :]  # q4 = yB (y position of base)

u1_vals_slip = sol_lagrange_slip.y[4, :]  # u1 = roll rate
u2_vals_slip = sol_lagrange_slip.y[5, :]  # u2 = yaw rate
u3_vals_slip = sol_lagrange_slip.y[6, :]  # u3 = xB velocity
u4_vals_slip = sol_lagrange_slip.y[7, :]  # u4 = yB velocity

# For the new solution
q1_vals_new = sol_lagrange_new.y[0, :]  # q1 = theta (roll angle)
q2_vals_new = sol_lagrange_new.y[1, :]  # q2 = r (yaw angle)
q3_vals_new = sol_lagrange_new.y[2, :]  # q3 = xB (x position of base)
q4_vals_new = sol_lagrange_new.y[3, :]  # q4 = yB (y position of base)

u1_vals_new = sol_lagrange_new.y[4, :]  # u1 = roll rate
u2_vals_new = sol_lagrange_new.y[5, :]  # u2 = yaw rate
u3_vals_new = sol_lagrange_new.y[6, :]  # u3 = xB velocity
u4_vals_new = sol_lagrange_new.y[7, :]  # u4 = yB velocity

# Compute the steer angle delta and its rate (d(delta)/dt) for both solutions
delta_vals = -2 * q1_vals  # Steer angle over time (original)
delta_dot_vals = np.gradient(delta_vals, time_points)  # Steer rate (time derivative)

delta_vals_new = -2 * q1_vals_new  # Steer angle over time (new)
delta_dot_vals_new = np.gradient(delta_vals_new, time_points_new)  # Steer rate (new)

# Convert all angles (q1, q2, delta) and angular rates (u1, u2, delta_dot) from radians to degrees
q1_vals_deg = np.degrees(q1_vals)
q2_vals_deg = np.degrees(q2_vals)
delta_vals_deg = np.degrees(delta_vals)
delta_dot_vals_deg = np.degrees(delta_dot_vals)

q1_vals_new_deg = np.degrees(q1_vals_new)
q2_vals_new_deg = np.degrees(q2_vals_new)
delta_vals_new_deg = np.degrees(delta_vals_new)
delta_dot_vals_new_deg = np.degrees(delta_dot_vals_new)

u1_vals_deg = np.degrees(u1_vals)
u2_vals_deg = np.degrees(u2_vals)

u1_vals_new_deg = np.degrees(u1_vals_new)
u2_vals_new_deg = np.degrees(u2_vals_new)

# Activate interactive mode
%matplotlib widget

import matplotlib.pyplot as plt
from mpl_interactions import ipyplot as iplt

# Plot each generalized coordinate q1 over time (in degrees)
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q1_vals_deg, ax=ax, label='Original')
iplt.plot(time_points_new, q1_vals_new_deg, ax=ax, label='New', linestyle='--')
ax.set_title("q1 (theta - Roll)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q1 (deg)")
ax.legend()
plt.show()

# Plot q2 (yaw angle) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q2_vals_deg, ax=ax, label='Original')
iplt.plot(time_points_new, q2_vals_new_deg, ax=ax, label='New', linestyle='--')
ax.set_title("q2 (r - Yaw)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q2 (deg)")
ax.legend()
plt.show()

# Plot q3 (xB position) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q3_vals, ax=ax, label='Original')
iplt.plot(time_points_new, q3_vals_new, ax=ax, label='New', linestyle='--')
ax.set_title("q3 (xB - X Position)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q3 (m)")
ax.legend()
plt.show()

# Plot q4 (yB position) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, q4_vals, ax=ax, label='Original')
iplt.plot(time_points_new, q4_vals_new, ax=ax, label='New', linestyle='--')
ax.set_title("q4 (yB - Y Position)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("q4 (m)")
ax.legend()
plt.show()

# Plot roll rate (u1) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, u1_vals_deg, ax=ax, label='Original')
iplt.plot(time_points_new, u1_vals_new_deg, ax=ax, label='New', linestyle='--')
ax.set_title("u1 (Roll Rate)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("u1 (deg/s)")
ax.legend()
plt.show()

# Plot yaw rate (u2) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, u2_vals_deg, ax=ax, label='Original')
iplt.plot(time_points_new, u2_vals_new_deg, ax=ax, label='New', linestyle='--')
ax.set_title("u2 (Yaw Rate)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("u2 (deg/s)")
ax.legend()
plt.show()

# Plot xB velocity (u3) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, u3_vals, ax=ax, label='Original')
iplt.plot(time_points_new, u3_vals_new, ax=ax, label='New', linestyle='--')
ax.set_title("u3 (xB Velocity)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("u3 (m/s)")
ax.legend()
plt.show()

# Plot yB velocity (u4) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, u4_vals, ax=ax, label='Original')
iplt.plot(time_points_new, u4_vals_new, ax=ax, label='New', linestyle='--')
ax.set_title("u4 (yB Velocity)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("u4 (m/s)")
ax.legend()
plt.show()

# Plot Steer angle (delta) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, delta_vals_deg, ax=ax, label='Original')
iplt.plot(time_points_new, delta_vals_new_deg, ax=ax, label='New', linestyle='--')
ax.set_title("Steer Angle (delta)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Steer Angle (deg)")
ax.legend()
plt.show()

# Plot Steer rate (delta_dot) over time
fig, ax = plt.figure(), plt.gca()
iplt.plot(time_points, delta_dot_vals_deg, ax=ax, label='Original')
iplt.plot(time_points_new, delta_dot_vals_new_deg, ax=ax, label='New', linestyle='--')
ax.set_title("Steer Rate (delta_dot)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Steer Rate (deg/s)")
ax.legend()
plt.show()




